In [ ]:
%reload_ext autoreload
%autoreload 2

import gif
import pylab as plt
import numpy as np
import matplotlib
from matplotlib.colors import LinearSegmentedColormap
from pythonperlin import perlin, extend2d

def remove_margins():
    """ Removes figure margins, keeps only plot area """
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    return


## Generate perlin noise

In [ ]:
%%time

""" Size of x and of the wireframe grid """
n = 10

""" Generate noise """
dens = 32
shape = (8,n+1,n+1)
p = perlin(shape, dens=dens, seed=0)

""" 
We need density only along time axis (axis=0). 
So we retain only each dens-th element along axes 1 and 2.
Prior to that, however, note that Perlin noise is zero at grid nodes. 
This is not good-looking visually, so first displace by dens//2.
"""
p = p[:,dens//2:][:,:,dens//2:]
p = p[:,::dens][:,:,::dens]

""" 
Use Perlin noise as a phase of a complex number z 
to generate a displacement noise for x and y. 
"""
z = np.exp(2j * np.pi * p)
px = z.real
py = z.imag


## Draw wireframe
<br>

- To generate a smoothly morphing wireframe we use Perlin noise (no octaves) to distort each grid node, then populate the grid with more lines along one direction (x).

In [ ]:
def plot_wireframe(i, px, py):
    xs = np.linspace(0,n,n+1)
    ys = np.linspace(0,n,n+1)
    x, y = np.meshgrid(xs, ys)
    np.random.seed(0)
    x += px[i]
    y += py[i]
    # Populate wires along x axis
    x = extend2d(x, 16, axis=1).T
    y = extend2d(y, 16, axis=1).T
    # Shrink x coordinate for better visual effect
    x = 1 + 0.8 * x
    
    fig = plt.figure(figsize=(6,6), facecolor="#000022")
    remove_margins()
    cmap = LinearSegmentedColormap.from_list("bw", ["dodgerblue", "white"])
    for i in range(x.shape[0]):
        color = cmap(i/x.shape[0])
        plt.plot(x[i], y[i], color=color)
    plt.xlim(-1,n+1)
    plt.ylim(-1,n+1)
    plt.tight_layout()
    return fig


""" Show one frame of the wireframe """
plot_wireframe(0, px, py)
plt.savefig("wireframe.jpg")

## Animate wireframe using 1st dimension as a time axis

In [ ]:
# Set the dots per inch resolution
gif.options.matplotlib["dpi"] = 180

# Decorate a plot function with @gif.frame
@gif.frame
def plot(i, px, py):
    plot_wireframe(i, px, py)

# Construct "frames"
frames = [plot(i, px, py) for i in range(0, len(px), 2)]

# Save "frames" to gif with a specified duration (milliseconds) between each frame
gif.save(frames, 'wireframe.gif', duration=200)